In [18]:
from query_rewrtiting import generate_rewritten_queries, model, tokenizer
from rag import RAG
from reranking import rerank_documents

def compare_retrieval_methods(query: str, k: int = 5):
    rag = RAG()
    
    print("=== Original Query ===")
    print(f"Query: {query}\n")
    
    # Normal querying results
    print("=== Results from Normal Querying ===")
    normal_results = rag.retrive_relevent_docs(query, top_k=k)
    for i, hit in enumerate(normal_results[0], 1):
        metadata = hit.entity.get('metadata')
        print(f"\n{i}. Luật: {metadata.get('luật')}")
        print(f"Điều {metadata.get('điều')}: {metadata.get('tên_điều')}")
        print(f"Nội dung: {metadata.get('chunk_content')}")
    
    # Query rewriting + reranking results
    print("\n=== Results from Query Rewriting + Reranking ===")
    rewritten_queries = generate_rewritten_queries(query, model, tokenizer)
    print("\nRewritten Queries:")
    for i, q in enumerate(rewritten_queries, 1):
        print(f"{i}. {q}")
    
    # Get results for each rewritten query
    all_results = []
    for q in rewritten_queries:
        results = rag.retrive_relevent_docs(q, top_k=k)
        all_results.extend(results[0])
    
    # Rerank results
    reranked_results = rerank_documents(query, all_results, top_k=k)
    print("\nReranked Results:")
    for i, (doc, score) in enumerate(reranked_results, 1):
        print(f"\n{i}. Relevance Score: {score}")
        print(doc)

if __name__ == "__main__":
    test_query = "Luật hôn nhân và gia đình là gì"
    compare_retrieval_methods(test_query, k=5)

Device set to use mps:0


=== Original Query ===
Query: Luật hôn nhân và gia đình là gì

=== Results from Normal Querying ===

1. Luật: LUẬT HÔN NHÂN VÀ GIA ĐÌNH
Điều Điều 95: Điều kiện mang thai hộ vì mục đích nhân đạo
Nội dung: 5. Chính phủ quy định chi tiết Điều này.

2. Luật: LUẬT NHÀ Ở
Điều Điều 194: Giải quyết tranh chấp về nhà ở
Nội dung: 5. Chính phủ quy định chi tiết khoản 3 Điều này.

3. Luật: LUẬT HÔN NHÂN VÀ GIA ĐÌNH
Điều Điều 54: Hòa giải tại Tòa án
Nội dung: Sau khi đã thụ lý đơn yêu cầu ly hôn, Tòa án tiến hành hòa giải theo quy định của pháp luật về tố tụng dân sự.

4. Luật: LUẬT ĐẤT ĐAI
Điều Điều 135: Nguyên tắc cấp Giấy chứng nhận quyền sử dụng đất, quyền sở hữu tài sản gắn liền với đất
Nội dung: .

5. Luật: LUẬT HÔN NHÂN VÀ GIA ĐÌNH
Điều Điều 58: Quyền, nghĩa vụ của cha mẹ và con sau khi ly hôn
Nội dung: Việc trông nom, chăm sóc, nuôi dưỡng, giáo dục con sau khi ly hôn được áp dụng theo quy định tại các điều 81, 82, 83 và 84 của Luật này.

=== Results from Query Rewriting + Reranking ===

Rew

In [14]:
# Mô hình cải tiến
import nltk
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

def evaluate_bleu(reference, candidate):
    reference_tokens = [ref.split() for ref in reference]
    candidate_tokens = candidate.split()
    smoothie = SmoothingFunction().method1
    
    bleu_score = sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=smoothie)
    return bleu_score

def evaluate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score("\n".join(reference), candidate)
    
    return {key: scores[key].fmeasure for key in scores}

if __name__ == "__main__":
    GPT_sentences = [
        "Điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều lu�",
        """
            1. Cơ quan có thẩm quyền trong bảo vệ an ninh mạng; giúp đỡ, tạo điều kiện cho cơ quan, tổ chức và người có trách nhiệm tiến hành các biện pháp bải bảo vảo vệ an ninhệ an ninh;

            2.
            3. Cười đi đi đi đi luềuềuềuềuề
        """
    ]
    Llama_sentences = [
        """Hình phạt cho tội trộm cắp tài sản (tội cưỡng đoạt tài sản) như đã quy định tại Điều 170 của Bộ luật Hình sự là:
            - Người phạm tội có thể bị phạt tiền từ 10.000.000 đồng đến 100.000.000 đồng.
            - Hoặc bị tịch thu một phần hoặc toàn bộ tài sản.
        """,
        """
            Theo Điều 2 của Luật An ninh mạng, An ninh mạng là sự bảo đảm hoạt động trên không gian mạng không gây phương hại đến an ninh quốc gia, trật tự, an toàn xã hội, quyền và lợi ích hợp pháp của cơ quan, tổ chức, cá nhân.
        """
    ]
    candidate_sentences = [
        """
            Hình phạt cho hành vi trộm cắp tài sản phụ thuộc vào giá trị tài sản bị trộm và các tình tiết liên quan. Theo Bộ luật Hình sự Việt Nam 2015 (sửa đổi, bổ sung 2017), trộm cắp tài sản có thể bị xử lý theo Điều 173 với các mức phạt như sau:

            1. Trường hợp xử phạt hành chính (nếu tài sản dưới 2 triệu đồng và không có tình tiết tăng nặng)
            Phạt tiền từ 1 triệu - 2 triệu đồng (theo Nghị định 144/2021/NĐ-CP).
            2. Trường hợp bị truy cứu trách nhiệm hình sự (Điều 173 BLHS 2015)
            Tùy theo giá trị tài sản và mức độ nghiêm trọng, hình phạt có thể bao gồm:

            Khoản 1 (Tài sản từ 2 triệu đồng trở lên hoặc dưới 2 triệu nhưng có tình tiết tăng nặng):

            Phạt cải tạo không giam giữ đến 03 năm, hoặc
            Phạt tù từ 06 tháng - 03 năm.
            Khoản 2 (Tài sản có giá trị từ 50 - 200 triệu đồng, hoặc phạm tội có tổ chức, chuyên nghiệp, tái phạm nguy hiểm...):

            Phạt tù từ 02 - 07 năm.
            Khoản 3 (Tài sản có giá trị từ 200 - 500 triệu đồng, hoặc có hành vi nguy hiểm khác...):

            Phạt tù từ 07 - 15 năm.
            Khoản 4 (Tài sản từ 500 triệu đồng trở lên hoặc gây hậu quả đặc biệt nghiêm trọng):

            Phạt tù từ 12 - 20 năm hoặc tù chung thân.
            Ngoài ra, người phạm tội có thể bị phạt tiền bổ sung từ 5 - 50 triệu đồng, hoặc bị cấm đảm nhiệm chức vụ, làm công việc nhất định sau khi mãn hạn tù.
        """,
        "An ninh mạng là sự bảo đảm hoạt động trên không gian mạng không gây phương hại đến an ninh quốc gia, trật tự, an toàn xã hội, quyền và lợi ích hợp pháp của cơ quan, tổ chức, cá nhân."
    ]
    
    results = []
    for model_name, references in zip(["Llama", "GPT"], [Llama_sentences, GPT_sentences]):
        for ref, candidate in zip(references, candidate_sentences):
            bleu = evaluate_bleu([ref], candidate)
            rouge = evaluate_rouge([ref], candidate)
            results.append({"Model": model_name, "Candidate": candidate, "BLEU Score": bleu, "ROUGE-1": rouge['rouge1'], "ROUGE-2": rouge['rouge2'], "ROUGE-L": rouge['rougeL']})
    
    df = pd.DataFrame(results)
    print(df)
    
    avg_scores = df.groupby("Model")[["BLEU Score", "ROUGE-1", "ROUGE-2", "ROUGE-L"]].mean().reset_index()
    print("\nAverage Scores:")
    print(avg_scores)

   Model                                          Candidate  BLEU Score  \
0  Llama  \n            Hình phạt cho hành vi trộm cắp t...    0.046241   
1  Llama  An ninh mạng là sự bảo đảm hoạt động trên khôn...    0.822734   
2    GPT  \n            Hình phạt cho hành vi trộm cắp t...    0.000900   
3    GPT  An ninh mạng là sự bảo đảm hoạt động trên khôn...    0.028100   

    ROUGE-1   ROUGE-2   ROUGE-L  
0  0.298805  0.212000  0.239044  
1  0.911765  0.910448  0.911765  
2  0.291417  0.040080  0.227545  
3  0.552239  0.181818  0.238806  

Average Scores:
   Model  BLEU Score   ROUGE-1   ROUGE-2   ROUGE-L
0    GPT    0.014500  0.421828  0.110949  0.233175
1  Llama    0.434487  0.605285  0.561224  0.575404


In [15]:
# Mô hình cơ bản
import nltk
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

def evaluate_bleu(reference, candidate):
    reference_tokens = [ref.split() for ref in reference]
    candidate_tokens = candidate.split()
    smoothie = SmoothingFunction().method1
    
    bleu_score = sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=smoothie)
    return bleu_score

def evaluate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score("\n".join(reference), candidate)
    
    return {key: scores[key].fmeasure for key in scores}

if __name__ == "__main__":
    GPT_sentences = [
        "Điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều luật trên các điều lu�",
        """
            Điều ước quốc tế mà nước Cộng hòa xã hội chủ nghĩa Việt Nam khóa XIII, kỳ họp thứ 7 thông qua ngày 20 tháng 11 năm 2012.
            4. Người phạm tội còn có thể bị phạt quản chế, cấm cư trú từ 01 năm đến 05 năm hoặc tịch thu một ph�
        """
    ]
    Llama_sentences = [
        "Hình phạt cho tội trộm cắp tài sản được quy định tại Điều 170 của Bộ luật Hình sự, theo đó người phạm tội có thể bị phạt tiền từ 10.000.000 đồng đến 100.000.000 đồng hoặc tịch thu một phần hoặc toàn bộ tài sản.",
        """
            Luật An ninh Mạng là một bộ luật quy định các quy định về bảo vệ và quản lý an ninh mạng của một quốc gia. Nó cung cấp định hướng cho việc phòng ngừa, phát hiện, ngăn chặn và xử lý hành vi xâm phạm an ninh mạng.

            Theo Luật An Ninh Mang, an ninh mạng được hiểu là sự bảo đảm hoạt động trên không gian mạng không gây phương hại đến an ninh quốc gia, trật tự, an toàn xã hội, quyền và lợi ích hợp pháp của cơ quan, tổ chức, cá nhân.
        """
    ]
    candidate_sentences = [
        """
            Hình phạt cho hành vi trộm cắp tài sản phụ thuộc vào giá trị tài sản bị trộm và các tình tiết liên quan. Theo Bộ luật Hình sự Việt Nam 2015 (sửa đổi, bổ sung 2017), trộm cắp tài sản có thể bị xử lý theo Điều 173 với các mức phạt như sau:

            1. Trường hợp xử phạt hành chính (nếu tài sản dưới 2 triệu đồng và không có tình tiết tăng nặng)
            Phạt tiền từ 1 triệu - 2 triệu đồng (theo Nghị định 144/2021/NĐ-CP).
            2. Trường hợp bị truy cứu trách nhiệm hình sự (Điều 173 BLHS 2015)
            Tùy theo giá trị tài sản và mức độ nghiêm trọng, hình phạt có thể bao gồm:

            Khoản 1 (Tài sản từ 2 triệu đồng trở lên hoặc dưới 2 triệu nhưng có tình tiết tăng nặng):

            Phạt cải tạo không giam giữ đến 03 năm, hoặc
            Phạt tù từ 06 tháng - 03 năm.
            Khoản 2 (Tài sản có giá trị từ 50 - 200 triệu đồng, hoặc phạm tội có tổ chức, chuyên nghiệp, tái phạm nguy hiểm...):

            Phạt tù từ 02 - 07 năm.
            Khoản 3 (Tài sản có giá trị từ 200 - 500 triệu đồng, hoặc có hành vi nguy hiểm khác...):

            Phạt tù từ 07 - 15 năm.
            Khoản 4 (Tài sản từ 500 triệu đồng trở lên hoặc gây hậu quả đặc biệt nghiêm trọng):

            Phạt tù từ 12 - 20 năm hoặc tù chung thân.
            Ngoài ra, người phạm tội có thể bị phạt tiền bổ sung từ 5 - 50 triệu đồng, hoặc bị cấm đảm nhiệm chức vụ, làm công việc nhất định sau khi mãn hạn tù.
        """,
        "An ninh mạng là sự bảo đảm hoạt động trên không gian mạng không gây phương hại đến an ninh quốc gia, trật tự, an toàn xã hội, quyền và lợi ích hợp pháp của cơ quan, tổ chức, cá nhân."
    ]
    
    results = []
    for model_name, references in zip(["Llama", "GPT"], [Llama_sentences, GPT_sentences]):
        for ref, candidate in zip(references, candidate_sentences):
            bleu = evaluate_bleu([ref], candidate)
            rouge = evaluate_rouge([ref], candidate)
            results.append({"Model": model_name, "Candidate": candidate, "BLEU Score": bleu, "ROUGE-1": rouge['rouge1'], "ROUGE-2": rouge['rouge2'], "ROUGE-L": rouge['rougeL']})
    
    df = pd.DataFrame(results)
    print(df)
    
    avg_scores = df.groupby("Model")[["BLEU Score", "ROUGE-1", "ROUGE-2", "ROUGE-L"]].mean().reset_index()
    print("\nAverage Scores:")
    print(avg_scores)

   Model                                          Candidate  BLEU Score  \
0  Llama  \n            Hình phạt cho hành vi trộm cắp t...    0.045476   
1  Llama  An ninh mạng là sự bảo đảm hoạt động trên khôn...    0.243472   
2    GPT  \n            Hình phạt cho hành vi trộm cắp t...    0.000900   
3    GPT  An ninh mạng là sự bảo đảm hoạt động trên khôn...    0.004423   

    ROUGE-1   ROUGE-2   ROUGE-L  
0  0.260692  0.188139  0.219959  
1  0.601942  0.598039  0.601942  
2  0.291417  0.040080  0.227545  
3  0.581560  0.158273  0.269504  

Average Scores:
   Model  BLEU Score   ROUGE-1   ROUGE-2   ROUGE-L
0    GPT    0.002662  0.436489  0.099177  0.248524
1  Llama    0.144474  0.431317  0.393089  0.410951
